In [ ]:
import numpy as np
import cv2
import os

import matplotlib as plt

import tensorflow as tf
import tensorflow_hub as hub

from tf_keras import layers 

# Use Keras 2.
version_fn = getattr(tf.keras, "version", None)
if version_fn and version_fn().startswith("3."):
  import tf_keras as keras
else:
  keras = tf.keras

In [ ]:
github_model_url = "https://github.com/RKPen/Project/tree/main/modelname"# Download the modelresponse = requests.get(github_model_url)

model_zip_path = "saved_model.zip"if response.status_code == 200:

    with open(model_zip_path, "wb") as f:

        f.write(response.content)# Unzip the modelimport zipfilewith zipfile.ZipFile(model_zip_path, "r") as zip_ref:

    zip_ref.extractall("my_saved_model")

In [ ]:
dataset_path = 'ENTER YOUR URL'

# Create an ImageDataGenerator for preprocessing images
data_generator = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0,1]
)

# Load and preprocess training data
data_flow = data_generator.flow_from_directory(
    dataset_path,
    target_size=(224, 224),  # Resize images to 224x224
    batch_size=32,
    class_mode='categorical',
)

# Print class indices
print("Class Indices:", data_flow.class_indices)


In [ ]:
model = keras.models.load_model(saved_model_path)

# Step 3: Make Predictions on Test Dataset# Get the predictions from the model
predictions = model.predict(data_flow)

# Convert predictions to class labels
predicted_classes = np.argmax(predictions, axis=1)

# Step 4: Evaluate the Model

# Get the true labels from the test data
true_labels = test_data_flow.classes  

# Actual class indices for the test set

# Calculate accuracy
accuracy = np.mean(predicted_classes == true_labels)  
# Accuracy as the score
print("Model Accuracy on Test Set:", accuracy)

# Optional: You can also use sklearn's classification_report for more detailed metrics
from sklearn.metrics import classification_report
print("Classification Report:")
print(classification_report(true_labels, predicted_classes, target_names=list(data_flow.class_indices.keys())))